<img src="./images/DLI_Header.png" style="width: 400px;">

# Classifying different types of Fruits based on whether rotten or not

## The Dataset

The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

We are going to be be training on a preloaded model and therefore use transer learning

In [1]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

In [2]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## Freeze Base Model

In [3]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

In [4]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))


# Add pooling layer or flatten layer
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)
#outputs = keras.layers.Dense(1)(x)
# Combine inputs and outputs to create model
#model = keras.Model(FIXME)
model = keras.Model(inputs, outputs)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 3078      
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [6]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Augment the Data

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

## Load Dataset

In [8]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('data/fruits/train/', 
                                        target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode="categorical")
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('data/fruits/valid/', 
                                       target_size=(224, 224),  
                                      color_mode='rgb', 
                                      class_mode="categorical")

Found 1182 images belonging to 6 classes.
Found 329 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [9]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)
#model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

Epoch 1/20
37/36 [==============================] - 28s 754ms/step - loss: 2.5433 - accuracy: 0.4805 - val_loss: 1.1749 - val_accuracy: 0.6596
Epoch 2/20
37/36 [==============================] - 20s 528ms/step - loss: 0.7627 - accuracy: 0.7699 - val_loss: 0.5560 - val_accuracy: 0.8085
Epoch 3/20
37/36 [==============================] - 20s 532ms/step - loss: 0.3715 - accuracy: 0.8756 - val_loss: 0.4067 - val_accuracy: 0.8784
Epoch 4/20
37/36 [==============================] - 19s 526ms/step - loss: 0.2518 - accuracy: 0.9129 - val_loss: 0.2056 - val_accuracy: 0.9422
Epoch 5/20
37/36 [==============================] - 19s 526ms/step - loss: 0.1765 - accuracy: 0.9433 - val_loss: 0.2475 - val_accuracy: 0.9240
Epoch 6/20
37/36 [==============================] - 20s 541ms/step - loss: 0.1334 - accuracy: 0.9492 - val_loss: 0.2066 - val_accuracy: 0.9240
Epoch 7/20
37/36 [==============================] - 19s 527ms/step - loss: 0.0957 - accuracy: 0.9653 - val_loss: 0.1849 - val_accuracy: 0.9544

## Unfreeze Model for Fine Tuning

In [10]:
# Unfreeze the base model
base_model.trainable = True


# Compile the model with a low learning rate
#model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),  # Very low learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy']
              )

In [12]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10)
#model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=10)

Epoch 1/10
18/36 [=============>................] - ETA: 19s - loss: 0.6360 - binary_accuracy: 0.9896

KeyboardInterrupt: 

## Evaluate the Model

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)

## Run the Assessment

<img src="./images/assess_task.png" style="width: 800px;">